In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
mydrive ="/content/drive/My Drive/Colab Notebooks/DataAnalytics"
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataAnalytics/DataScientist.csv')
df = df.iloc[:,2:]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(["data", "experience", "including", "required", "job"])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df['Job Description'].values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['hopper', 'hopper', 'mission', 'make', 'booking', 'travel', 'faster', 'easier', 'transparent', 'leveraging', 'power', 'comes', 'combining', 'massive', 'amounts', 'machine', 'learning', 'build', 'world', 'fastest', 'growing', 'travel', 'app', 'one', 'enables', 'customers', 'save', 'money', 'travel', 'cad']


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 3), (12, 1), (13, 1), (14, 5), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 6)]


In [ ]:
from pprint import pprint
# number of topics
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.008*"business" + 0.006*"work" + 0.006*"team" + 0.005*"skills" + 0.005*"analysis" + 0.005*"solutions" + 0.005*"years" + 0.005*"development" + 0.005*"ability" + 0.004*"analytics"'), (1, '0.009*"business" + 0.009*"work" + 0.005*"science" + 0.005*"years" + 0.005*"development" + 0.005*"analysis" + 0.005*"skills" + 0.005*"team" + 0.004*"information" + 0.004*"requirements"'), (2, '0.009*"work" + 0.007*"team" + 0.006*"skills" + 0.006*"business" + 0.005*"development" + 0.005*"science" + 0.005*"years" + 0.004*"ability" + 0.004*"strong" + 0.004*"new"')]


In [ ]:
#import pyLDAvis.gensim
#from pyLDAvis import gensim
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('ldavis_prepared_explore_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_explore_'+ str(num_topics) +'.html')
LDAvis_prepared

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.005718  0.003964       1        1  37.381923
0      0.001073 -0.006305       2        1  33.311820
1     -0.006791  0.002341       3        1  29.306257, topic_info=            Term         Freq        Total Category  logprob  loglift
29      business  9408.000000  9408.000000  Default  30.0000  30.0000
1452  employment  2536.000000  2536.000000  Default  29.0000  29.0000
741      support  3922.000000  3922.000000  Default  28.0000  28.0000
237         work  9975.000000  9975.000000  Default  27.0000  27.0000
11      analysis  5285.000000  5285.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
408      develop   979.773225  3481.833966   Topic3  -5.9315  -0.0406
505    solutions  1071.109017  4493.992450   Topic3  -5.8423  -0.2067
246    analytics  1021.529281  4455.989418   Topic3  -5.8897  -0.2456
204    technical   995.397469  4349.474434   Topic3  -5.9156  -0.2473
463          new   978.582267  4118.508036   Topic3  -5.9327  -0.2098

[323 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
20808      3  0.878612  abigdata
0          1  0.399736   ability
0          2  0.362374   ability
0          3  0.237897   ability
20115      1  0.409999  abinitio
...      ...       ...       ...
20770      1  0.412133   wrapper
20770      2  0.824266   wrapper
241        1  0.383482     years
241        2  0.319596     years
241        3  0.296932     years

[674 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])